### LangChain and OpenAI LLM Quick Demo

This demo showcases how to use LangChain to build useful large language model applications. LangChain is a framework that allows us to interact with Large Language Models (LLMs) like OpenAI GPTs.

First, install the required modules.

Always best to upgrade since `langchain` is rapidliy developing.


In [33]:
! pip install langchain --upgrade
! pip install openai --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.4/515.4 kB 498.7 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.135
    Uninstalling langchain-0.0.135:
      Successfully uninstalled langchain-0.0.135


Then, import the packages to use.

Please create your [OpenAI](https://openai.com/) key before proceeding. 

In [35]:
import os
import textwrap
from langchain.llms import OpenAI

query = input("OpenAI key: ")
os.environ["OPENAI_API_KEY"] = query

#### Demo 1: A Simple Question and Answering Example

The simplest example is a question and answering application. To build the demo, execute the following: 

- Load OpenAI Large Language Model (`llm`). Temperature is a measure of uncertainty or randomness of answers. Higher temperature means more randomness. In other words, temp = 0 means the model will always give the same answer. Temp = 1 means the model will give answers with more variance. Let's set it to 0.5.
- Provide a text query
- Request the Large Language Model to process the query
- Print the result

In [36]:
llm = OpenAI(temperature=0.5)
text = "What are the 10 best places to visit?"
print(llm(text))



1. Paris, France
2. Rome, Italy
3. London, England
4. New York City, USA
5. Barcelona, Spain
6. Tokyo, Japan
7. Amsterdam, Netherlands
8. Bangkok, Thailand
9. Istanbul, Turkey
10. Rio de Janeiro, Brazil


#### Demo 2: Using a Prompt Template

A meaningful interaction with a LLM requires a good prompt to elicit a good answer. In this example a prompt template is provided to elicit a good answer from the LLM. 

In [37]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(
    input_variables=["to_do_what"],
    template="What are the 10 best places to {to_do_what}?",
)
formatted_prompt = prompt.format(to_do_what="eat")
print(f"Formatted prompt: {formatted_prompt}")

Formatted prompt: What are the 10 best places to eat?


#### Demo 3: A Simple Chain

LLM is not very useful unless we can interact with it using prompt. In the same way, a prompt is not valuable unless we can use it to interact with a LLM. A chain puts them together. Below is a simple chain.

Chain: Prompt &rarr; LLM 

Then, the chain can now be queried.

`chain.run(<to_do_what>`

In [40]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)
formatted_prompt = prompt.format(to_do_what="...")
query = input(formatted_prompt)
formatted_prompt = prompt.format(to_do_what=query)
print(f"Human: {formatted_prompt}")
results = chain.run(query).split('\n')
print(f"AI: ")
for result in results:
    print(result)

Human: What are the 10 best places to eat street food?
AI: 


1. Bangkok, Thailand
2. Istanbul, Turkey
3. Mexico City, Mexico
4. Penang, Malaysia
5. Hoi An, Vietnam
6. New Delhi, India
7. Hong Kong
8. Marrakesh, Morocco
9. Tokyo, Japan
10. Seoul, South Korea


#### Demo 4: Enabling Long-Term Memory 

A conversation has context. Sometimes, the context has a reference to a discussion long time ago. Without the ability to remember something said a while back, the conversation can easily become frustrating. Thus, memory is important. In this example, a memory is incorporated in the chain.

Below is a conversation with context.

In [41]:
from langchain import OpenAI, ConversationChain

llm = OpenAI(temperature=0.5)
conversation = ConversationChain(llm=llm)

conversation.predict(input="Hi! I want to talk to you. What is your name?")

" Hi there! My name is AI-01. It's nice to meet you. How can I help you today?"

##### Let's Build a Conversation Chain

The human can ask questions all about a topic. Assume that there is context in the conversation. To end the conversation, the human can say "bye". 

In [45]:
while True:
    input_prompt = "Human: "
    query = input(input_prompt)

    if query.lower() == "bye":
        print("AI: Bye!")
        break

    text = f"{input_prompt}: {query}"
    print(textwrap.fill(text, width=80))
    text = f"AI: {conversation.predict(input=query)}"
    print(textwrap.fill(text, width=80))

Human: : I want to travel the world. I want to go to southeast asia. please help
me prepare travel plans.
AI:  Absolutely! Southeast Asia is a great place to visit. What kind of places
are you interested in seeing in Southeast Asia? Are you looking for cities,
beaches, mountains, or something else?
Human: : beaches
AI:   Got it. Let me see what I can find for you. I'm sure I can come up with
some great beach destinations with plenty of activities. Give me a few minutes
to compile some ideas and I'll get back to you shortly.
Human: : do you have anything in mind
AI:    Absolutely! I've compiled a few ideas for you. Here are some of the top
beach destinations in Southeast Asia that I recommend: Boracay in the
Philippines, Koh Samui in Thailand, Tanjung Aru Beach in Malaysia, and Nusa Dua
in Indonesia. Do any of these sound like places you'd like to visit?
Human: : i heard boracay is great
AI:    Yes, Boracay is a wonderful destination. I can help you plan out your
itinerary and book any 

#### Demo 5: Using Agent and Tools to Determine what Actions to Take to Solve a Problem

Sometimes, a problem can not be solved in one step. Additional tools may also needed in order to solve the problem. In this demo, the LLM uses an agent to determine what actions to take and what tools to use.

The focus is on math problems. `llm-math` tool is used. More info about [tools](https://langchain.readthedocs.io/en/latest/modules/agents/tools.html) and [agents](https://langchain.readthedocs.io/en/latest/modules/agents/agents.html) are available in the LangChain documentation.

In [46]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent

In [47]:
# First, let's load the language model we're going to use to control the agent.
llm = OpenAI(temperature=0.0)

# Next, let's load some tools to use. 
# Note that the `llm-math` tool uses an LLM, so we need to pass that in.
tools = load_tools(["llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, 
# and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

# Now let's test it out!

query = input("Human: ")
text = f"Human: {query}"
agent.run(query)



> Entering new AgentExecutor chain...
 I need to figure out how many oranges are in the basket
Action: Calculator
Action Input: 10 - 5 + 8
Observation: Answer: 13

Thought: I now know the final answer
Final Answer: There are 13 oranges in the basket.

> Finished chain.


'There are 13 oranges in the basket.'